### prompt 1


Gere em python um algoritmo que observe os seguintes requisitos:
1. crie um algoritmo de árvore de decisão from scratch;
2. não utilize bibliotecas como scikit learn ou quaisquer outras que implementem tal algoritmo;
3. utilize a heurística top-down;
4. utilize uma estratégia recursiva e gulosa (greedy);
5. utilize quando necessário bibliotecas de manipulaçao de dados como Pandas e Numpy;
6. os atributos poderão ser tanto categóricos, como numéricos (inteiro e float);


### Resposta ao prompt 1

Aqui está uma implementação de um algoritmo de árvore de decisão from scratch que atende os requisitos:

Top-down: O algoritmo começa da raiz e segue até as folhas.
Recursivo e guloso (greedy): Ele escolhe, em cada nó, a melhor divisão (feature) com base na redução de entropia ou ganho de informação, e faz isso recursivamente para cada subdivisão.
A implementação usa Numpy para manipulação de dados e cálculos de ganho de informação, e Pandas para manipulação de datasets.

In [1]:
import numpy as np
import pandas as pd

class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def fit(self, X:pd.DataFrame, y):
        dataset = X.copy()
        dataset['label'] = y
        self.tree = self._build_tree(dataset)

    def predict(self, X:pd.DataFrame):
        return X.apply(self._predict_row, axis=1)

    def _entropy(self, y):
        # Calcula a entropia dos rótulos
        _, class_counts = np.unique(y, return_counts=True)
        probabilities = class_counts / len(y)
        return -np.sum(probabilities * np.log2(probabilities))

    def _information_gain(self, left_y, right_y, parent_entropy):
        # Calcula o ganho de informação após uma divisão
        left_weight = len(left_y) / (len(left_y) + len(right_y))
        right_weight = len(right_y) / (len(left_y) + len(right_y))
        child_entropy = (left_weight * self._entropy(left_y) +
                         right_weight * self._entropy(right_y))
        return parent_entropy - child_entropy

    def _best_split(self, dataset):
        # Encontra a melhor divisão
        best_gain = 0
        best_feature = None
        best_split_value = None
        parent_entropy = self._entropy(dataset['label'])
        
        for feature in dataset.columns[:-1]:
            values = dataset[feature].unique()
            for value in values:
                left_split = dataset[dataset[feature] <= value]
                right_split = dataset[dataset[feature] > value]
                
                if len(left_split) == 0 or len(right_split) == 0:
                    continue
                
                gain = self._information_gain(left_split['label'], right_split['label'], parent_entropy)
                
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_split_value = value
        
        return best_feature, best_split_value, best_gain

    def _build_tree(self, dataset:pd.DataFrame, depth=0):
        # Função recursiva para construir a árvore de decisão
        X : pd.DataFrame = dataset.drop(columns=['label'])
        y : np.array = dataset['label']
        
        # Critérios de parada
        if len(np.unique(y)) == 1:
            return np.unique(y)[0]  # Folha com uma única classe
        if len(X) < self.min_samples_split or (self.max_depth is not None and depth >= self.max_depth):
            return y.mode()[0]  # Folha com a classe majoritária
        
        # Melhor divisão
        best_feature, best_split_value, best_gain = self._best_split(dataset)
        
        if best_gain == 0:
            return y.mode()[0]  # Folha com a classe majoritária
        
        left_split = dataset[dataset[best_feature] <= best_split_value]
        right_split = dataset[dataset[best_feature] > best_split_value]
        
        # Nós filhos
        node = {
            'feature': best_feature,
            'value': best_split_value,
            'left': self._build_tree(left_split, depth + 1),
            'right': self._build_tree(right_split, depth + 1)
        }
        
        return node

    def _predict_row(self, row):
        node = self.tree
        while isinstance(node, dict):
            if row[node['feature']] <= node['value']:
                node = node['left']
            else:
                node = node['right']
        return node


**Detalhes do Algoritmo:**
- Heurística Top-Down: O algoritmo começa dividindo o dataset a partir da raiz (nó inicial), calculando a entropia de cada feature para determinar qual delas traz maior ganho de informação e faz a divisão baseada nesse cálculo. Esse processo é repetido recursivamente.

- Estratégia Recursiva e Gulosa: A divisão de cada nó é escolhida de forma gulosa, isto é, para cada nó, o algoritmo tenta encontrar a melhor divisão naquele momento, sem considerar o impacto da escolha no futuro.

**Parâmetros:**
`max_depth`: Limita a profundidade da árvore para evitar overfitting.
`min_samples_split`: Define o número mínimo de amostras para permitir a divisão do nó.

**Exemplo de uso:**

In [2]:
# Exemplo de uso com dataset fictício
data = {'feature1': [1, 2, 2, 3, 3],
        'feature2': [1, 1, 2, 2, 3],
        'label': [0, 0, 1, 1, 1]}
df = pd.DataFrame(data)
X = df.drop(columns=['label'])
y = df['label']

tree = DecisionTree(max_depth=3)
tree.fit(X, y)

# Predições
predictions = tree.predict(X)
print(predictions)

0    0
1    0
2    1
3    1
4    1
dtype: int64


### prompt 2

Implemente, novamente sem utilizar bibliotecas como scikit learn, um algoritmo que retorne a matriz de confusão.

### Resposta prompt 2

In [3]:
import numpy as np
import pandas as pd

def confusion_matrix(y_true, y_pred, labels=None):
    """
    Calcula a matriz de confusão para os rótulos verdadeiros e preditos.

    Parâmetros:
    y_true: array-like de rótulos verdadeiros.
    y_pred: array-like de rótulos preditos.
    labels: array-like dos rótulos únicos (opcional). Se não fornecido, será inferido a partir de y_true.

    Retorna:
    Uma matriz de confusão (DataFrame).
    """
    if labels is None:
        labels = np.unique(y_true)
    
    # Inicializa a matriz de confusão com zeros
    matrix = pd.DataFrame(np.zeros((len(labels), len(labels)), dtype=int), index=labels, columns=labels)
    
    # Preenche a matriz com as contagens
    for true_label, predicted_label in zip(y_true, y_pred):
        matrix.at[true_label, predicted_label] += 1
    
    return matrix

### prompt 3
Agora, gere funções em Python para calcular as seguintes métricas: acurácia, recall, precisão. Novamente, não utilize bibliotecas como scikit learn.

### Resposta ao prompt 3

In [4]:
import numpy as np

def accuracy(conf_matrix):
    """
    Calcula a acurácia a partir da matriz de confusão.
    
    Parâmetros:
    conf_matrix: DataFrame (ou array) representando a matriz de confusão.
    
    Retorna:
    Acurácia (float).
    """
    total_correct = np.trace(conf_matrix)  # Soma dos elementos da diagonal principal (acertos)
    total_samples = conf_matrix.sum().sum()  # Soma de todos os elementos da matriz (total de amostras)
    return total_correct / total_samples

def precision(conf_matrix):
    """
    Calcula a precisão para cada classe a partir da matriz de confusão.
    
    Parâmetros:
    conf_matrix: DataFrame (ou array) representando a matriz de confusão.
    
    Retorna:
    Um dicionário com a precisão de cada classe.
    """
    precisions = {}
    for label in conf_matrix.columns:
        true_positive = conf_matrix.at[label, label]  # Verdadeiros positivos (diagonal)
        predicted_positive = conf_matrix[label].sum()  # Soma da coluna (todos os preditos como essa classe)
        precisions[label] = true_positive / predicted_positive if predicted_positive > 0 else 0
    return precisions

def recall(conf_matrix):
    """
    Calcula o recall para cada classe a partir da matriz de confusão.
    
    Parâmetros:
    conf_matrix: DataFrame (ou array) representando a matriz de confusão.
    
    Retorna:
    Um dicionário com o recall de cada classe.
    """
    recalls = {}
    for label in conf_matrix.index:
        true_positive = conf_matrix.at[label, label]  # Verdadeiros positivos (diagonal)
        actual_positive = conf_matrix.loc[label].sum()  # Soma da linha (todos os reais dessa classe)
        recalls[label] = true_positive / actual_positive if actual_positive > 0 else 0
    return recalls


## Prompt 4
Agora, gere um algoritmo, novamente sem utilizar scikit learn, para calcular o score f1.

### Resposta ao Prompt 4
O F1 Score é a média harmônica entre precisão (precision) e recall. Ele é usado para medir o equilíbrio entre essas duas métricas e é especialmente útil em problemas de classificação desbalanceados.
Aqui está um algoritmo para calcular o F1 Score para cada classe, com base nas funções de precisão e recall que já implementamos:

In [5]:
def f1_score(conf_matrix):
    """
    Calcula o F1 Score para cada classe a partir da matriz de confusão.
    
    Parâmetros:
    conf_matrix: DataFrame (ou array) representando a matriz de confusão.
    
    Retorna:
    Um dicionário com o F1 Score de cada classe.
    """
    precisions = precision(conf_matrix)  # Calcula precisão
    recalls = recall(conf_matrix)  # Calcula recall
    f1_scores = {}

    for label in conf_matrix.index:
        p = precisions[label]
        r = recalls[label]
        if (p + r) > 0:
            f1_scores[label] = 2 * (p * r) / (p + r)
        else:
            f1_scores[label] = 0  # Evita divisão por zero quando p + r = 0

    return f1_scores


### Formatando o código da rotina principal
**É esse trecho de código que executará a árvore de decisão.

In [6]:
def imprime_dict(dict_instance:dict, metrica:str)->None:
    print(f"métrica: {metrica}")
    for key,value in dict_instance.items():
        print(f"rótulo: {key}, pontuação: {value}")
    return
def pre_processing_titanic(df:pd.DataFrame)->pd.DataFrame:
    df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
    # Handle missing values
    df['Age'].fillna(df['Age'].median(), inplace=True)
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
    df['Fare'].fillna(df['Fare'].median(), inplace=True)
    # Feature engineering
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    return df
def main()->None:
    df : pd.DataFrame = pd.read_csv("titanic.csv")
    tree : DecisionTree = DecisionTree(max_depth=5)
    # Remove unnecessary columns
    df = pre_processing_titanic(df)
    # Prepare features and target
    X = df.drop('Survived', axis=1)
    y = df['Survived']
    tree.fit(X,y)
    predictions = tree.predict(X)
    print(predictions)
    conf_matrix_df : pd.DataFrame = confusion_matrix(y,predictions)
    accuracy_value : float = accuracy(conf_matrix=conf_matrix_df)
    recall_dict : dict = recall(conf_matrix=conf_matrix_df)
    precision_dict :dict = precision(conf_matrix=conf_matrix_df)
    f1_score_dict : dict = f1_score(conf_matrix=conf_matrix_df)
    print(f"matriz de confusão: {conf_matrix_df}")
    print(f"acurácia: {accuracy_value}")
    imprime_dict(recall_dict,"recall")
    imprime_dict(precision_dict,"precision")
    imprime_dict(f1_score_dict,"f1 score")
if __name__=='__main__':
    main()

C:\Users\fcfri\AppData\Local\Temp\ipykernel_14052\2333426113.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
C:\Users\fcfri\AppData\Local\Temp\ipykernel_14052\2333426113.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For 

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    0
890    0
Length: 891, dtype: int64
matriz de confusão:      0    1
0  498   51
1   81  261
acurácia: 0.8518518518518519
métrica: recall
rótulo: 0, pontuação: 0.907103825136612
rótulo: 1, pontuação: 0.7631578947368421
métrica: precision
rótulo: 0, pontuação: 0.8601036269430051
rótulo: 1, pontuação: 0.8365384615384616
métrica: f1 score
rótulo: 0, pontuação: 0.8829787234042553
rótulo: 1, pontuação: 0.7981651376146789


## Análise crítica do código gerado: 
1. pontos fortes,
2. potenciais problemas/diferenças diante das implementações discutidas em aula
3. opções de projeto realizadas pelo ChatGPT e seu impacto na implementação
4. sugestões de melhorias